In [5]:
!pip install  tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [1]:
import pandas as pd
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType

In [2]:
csv_file_path = 'sales_report.csv'

In [3]:
llm = ChatOpenAI(openai_api_base="http://localhost:1234/v1", openai_api_key="lm_studio", model="local-model")

c:\Users\deepa\.conda\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
print("\nCreating CSV Agent...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True, 
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # Allow the agent to execute python code
    handle_parsing_errors=True # Handle cases where the LLM output is not perfectly formatted

)
print("Agent is ready.")

In [16]:
AGENT_PREFIX = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you.
You have access to a single tool: `python_repl_ast`.
Use this to process the dataframe and return the final answer.

Your response MUST be in the following format:

Question: the user's question
Thought: I need to use pandas to find the answer. I will write the code to do that.
Action: python_repl_ast
Action Input: a single, valid line of pandas code to run

Example:
Question: how many rows are there?
Thought: I need to count the rows in the dataframe.
Action: python_repl_ast
Action Input: df.shape[0]
"""

print("\nCreating CSV Agent with a guiding prefix...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    prefix=AGENT_PREFIX
)
print("Agent is ready.")


Creating CSV Agent with a guiding prefix...
Agent is ready.


c:\Users\deepa\.conda\envs\rag\lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:242: UserWarning: Received additional kwargs {'allow_dangerous_code': True, 'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [17]:
query1 = "What was the total sale amount for the North region?"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])



Question: What was the total sale amount for the North region?


> Entering new AgentExecutor chain...
Thought: I need to find the sum of the sales amounts for the North region.
Action: python_repl_ast
Action Input: df[df['Region'] == 'North']['Sale Amount'].sum() 17000Thought: I now know the final answer
Final Answer: 17000 


> Finished chain.
Answer: 17000


In [18]:
query1 = "how many laptops sold in south region"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])


Question: how many laptops sold in south region


> Entering new AgentExecutor chain...
Thought: I need to count the number of rows where the 'Region' is 'South' and the product is 'Laptop'. 
Action: python_repl_ast
Action Input: df[df['Region'] == 'South'][df['Product'] == 'Laptop']['Units Sold'].sum() 5

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Thought: I now know the final answer
Final Answer: 5 


> Finished chain.
Answer: 5
